In [72]:
import pandas as pd
import numpy as np

# Parâmetros de cada loja
abs = pd.read_excel(r"C:\Users\wallace.souza\Desktop\Scripts\Abastecimento\Parametro.xlsx")

# Planilha de cadastro para mesclar os produtos de mercearia

abs1 = pd.read_excel(r"C:\Users\wallace.souza\Desktop\Scripts\Abastecimento\Parametro.xlsx", sheet_name=1)



In [73]:
abs['CURVA ATUAL'] = abs['CURVA ATUAL'].replace(0, 'C')


def ajustar_quantidade(row):
    while (row['QTD ESTQ MÁXIMO'] <= row['QTD ESTQ MÍNIMO']).any():
        row.loc[row['QTD ESTQ MÁXIMO'] <= row['QTD ESTQ MÍNIMO'], 'QTD ESTQ MÁXIMO'] += row['UNID']

def ajustar_quantidade2(row2):
    while (row2['QTD ESTQ MÍNIMO'] < (row2['QTD ESTQ MÁXIMO'] - row2['QTD ESTQ MÍNIMO'])).any():
     row2.loc[row2['QTD ESTQ MÍNIMO'] < (row2['QTD ESTQ MÁXIMO'] - row2['QTD ESTQ MÍNIMO']), 'QTD ESTQ MÍNIMO'] += 1

def dias_cobertura(x):
  x['DIAS COBERT MÍNIMO'] = x['CURVA ATUAL'].apply(lambda curva: 7 if curva == 'A' else (12 if curva == 'B' else 15))
  x['DIAS COBERT MÁXIMO'] = x['CURVA ATUAL'].apply(lambda curva: 10 if curva == 'A' else (15 if curva == 'B' else 20))
  return x


In [74]:
abs = abs.merge(abs1, on='SEQPRODUTO', how='left')
abs = abs[abs['SETOR']=='MERCEARIA']

abs = abs.sort_values(by=['LOJA','DESCRICAO PRODUTO', 'FRENTE'], ascending=[True, True, False])
abs = abs.drop_duplicates(subset=['LOJA', 'DESCRICAO PRODUTO'], keep='first')

abs = abs[['LOJA', 'SEQPRODUTO','UNID', 'FRENTE', 'TOTAL UND','CURVA ATUAL', 'QTD ESTQ MÍNIMO',
       'QTD ESTQ MÁXIMO']]

abs = abs[['LOJA', 'SEQPRODUTO','UNID', 'FRENTE', 'TOTAL UND','CURVA ATUAL', 'QTD ESTQ MÍNIMO',
       'QTD ESTQ MÁXIMO']]

In [75]:
abs ['QTD ESTQ MÁXIMO'] = abs['UNID']
abs['QTD ESTQ MÍNIMO'] = abs['TOTAL UND']
abs['DIAS COBERT MÍNIMO'] = abs['DIAS COBERT MÍNIMO'] = np.nan
abs['DIAS COBERT MÁXIMO'] = abs['DIAS COBERT MÁXIMO'] = np.nan

In [76]:
abs = abs.drop_duplicates(subset=['LOJA', 'SEQPRODUTO'], keep='first')


In [77]:
dias_cobertura(abs)
ajustar_quantidade(abs)

In [78]:

abs = abs[['LOJA', 'SEQPRODUTO','QTD ESTQ MÍNIMO', 'QTD ESTQ MÁXIMO', 'DIAS COBERT MÍNIMO',
       'DIAS COBERT MÁXIMO']]

abs = abs[~abs['SEQPRODUTO'].isin([41102,1257,3938,31013,52])]


abs.columns= [['LOJA', 'SEQPRODUTO', 'QTD ESTQ MINIMO', 'QTD ESTQ MAXIMO',
       'DIAS COBERT MINIMO', 'DIAS COBERT MAXIMO']]

In [79]:
abs.to_csv('Ajuste de Parâmetro.csv', sep=';', index=False)
